In [4]:
!pip install Faker

In [5]:
import random
from faker import Faker
from datetime import datetime, timedelta
import google.generativeai as genai

In [6]:

def gerar_personalidade(informacoes):
    """Gera uma descrição de personalidade usando o modelo Gemini."""

    prompt = f"""
    Com base nas seguintes informações, gere uma descrição de personalidade detalhada e realista:
    Caso a pessoa tenha nome Social utilize o nome social para a descrição da personalidade

    **Localização:**
        - Cidade: {informacoes['Localização']['Cidade']}
        - Estado: {informacoes['Localização']['Estado']}

    **Dados Pessoais:**
        - Nome: {informacoes['Dados Pessoais']['Nome']}
        - Idade: {informacoes['Dados Pessoais']['Idade']}
        - Profissão: {informacoes['Dados Pessoais']['Profissão']}
        - Nome Social: {informacoes['Dados Pessoais']['Nome Social']}

    **Dados Financeiros:**
        - Renda Mensal: {informacoes['Dados Financeiros']['Renda Mensal']}
        - Patrimônio: {informacoes['Dados Financeiros']['Patrimônio']}

    **Dados Adicionais:**
        - Data de Nascimento: {informacoes['Dados Adicionais']['Data de Nascimento']}
        - Gênero: {informacoes['Dados Adicionais']['Gênero']}
        - Estado Civil: {informacoes['Dados Adicionais']['Estado Civil']}
    """

    try:
        # Configuração da API (substitua pela sua chave real)
        genai.configure(api_key='')

        model = genai.GenerativeModel(
                model_name='gemini-pro')
        # Geração do conteúdo (modelo especificado aqui)
        response = model.generate_content(
            prompt,
            generation_config={
                'temperature': 0.7,
                'candidate_count':1,
                'max_output_tokens': 200
            }
        )

        # Extração da resposta
        if response.text is not None and response.text.strip():
            return response.text
        else:
            print("A API retornou uma resposta vazia ou inválida.")

    except Exception as e:
        print(f"Erro na API do Gemini: {e}")
        return "Erro ao gerar personalidade"

In [7]:
def gerar_nome_social(nome_completo,genero):
    """Gera um nome Social caso necessário."""

    prompt = f"""
    Identifique o gênero esperado a partir do nome enviado.
    Caso o gênero esperado seja igual ao gênero enviado retorne Nenhum, caso seja diferente retorne um nome social para a pessoa.
    Não crie um nome social parecido com o nome original e retorne apenas o nome social.

    - Nome: {nome_completo}
    - Gênero: {genero}
    """

    try:
        # Configuração da API (substitua pela sua chave real)
        genai.configure(api_key='')

        model = genai.GenerativeModel(
                model_name='gemini-pro')
        # Geração do conteúdo (modelo especificado aqui)
        response = model.generate_content(
            prompt,
            generation_config={
                'temperature': 0.7,
                'candidate_count':1,
                'max_output_tokens': 200
            }
        )

        # Extração da resposta
        if response.text is not None and response.text.strip():
            return response.text
        else:
            print("A API retornou uma resposta vazia ou inválida.")

    except Exception as e:
        print(f"Erro na API do Gemini: {e}")
        return "Erro ao gerar nome"

In [8]:
def obter_informacoes_ficticias(
        idade_min=18,
        idade_max=80,
        renda_min=1000,
        renda_max=20000,
        patrimonio_min=-10000,
        patrimonio_max=500000,
        num_compras=5,
):
    """Gera informações fictícias detalhadas e realistas."""

    fake = Faker("pt_BR")

    # Dados de Localização
    cidade = fake.city()
    estado = fake.state()
    cep = fake.postcode()
    logradouro = fake.street_address()

    # Dados Pessoais (com geração de email mais realista)
    nome_completo = fake.name()
    primeiro_nome = nome_completo.split()[0].lower()
    sobrenome = nome_completo.split()[-1].lower()
    idade = random.randint(idade_min, idade_max)
    email = f"{primeiro_nome}.{sobrenome}@{fake.free_email_domain()}"
    ddd = fake.msisdn()[3:5]
    telefone = fake.phone_number()
    cpf = fake.cpf()
    profissao = fake.job()

    # Redes Sociais
    redes_sociais = {}
    if random.random() < 0.8:
        redes_sociais["Facebook"] = f"https://www.facebook.com/{fake.user_name()}"
    if random.random() < 0.6:
        redes_sociais["Instagram"] = f"https://www.instagram.com/{fake.user_name()}"
    if random.random() < 0.4:
        redes_sociais["Twitter"] = f"https://twitter.com/{fake.user_name()}"

    # Dados Financeiros
    renda_mensal = round(random.uniform(renda_min, renda_max), 2)
    patrimonio = round(random.uniform(patrimonio_min, patrimonio_max), 2)

    # Lógica para dívidas e nome sujo
    dividas = 0
    nome_sujo = False
    if patrimonio < 0:
        dividas = round(random.uniform(0, abs(patrimonio)), 2)
        nome_sujo = True
    elif random.random() < 0.1:
        dividas = round(random.uniform(0, renda_mensal * 3), 2)
        nome_sujo = True

    # Dados Adicionais
    data_nascimento = fake.date_of_birth(minimum_age=idade_min, maximum_age=idade_max)
    genero = random.choice(["Masculino", "Feminino"])
    estado_civil = random.choice(["Solteiro(a)", "Casado(a)", "Divorciado(a)", "Viúvo(a)"])

    #Nome social
    nome_social = gerar_nome_social(nome_completo,genero)

    # Dados Bancários
    banco = fake.bank_country()
    agencia = fake.bothify(text="####-#")
    conta = fake.bothify(text="######-#")

    # Histórico de Compras
    hoje = datetime.now()
    historico_compras = []
    for _ in range(num_compras):
        data_compra = hoje - timedelta(days=random.randint(1, 365))
        produto = fake.random_element(
            elements=(
                "Smartphone",
                "Notebook",
                "Roupas",
                "Alimentos",
                "Livro",
                "Passagem aérea",
            )
        )
        valor = round(random.uniform(10, 1000), 2)
        historico_compras.append(
            {"Data": data_compra.strftime("%d/%m/%Y"), "Produto": produto, "Valor": f"R$ {valor:.2f}"}
        )

    # Criação do dicionário de informações (ANTES de gerar a personalidade)
    informacoes = {
        "Localização": {
            "Cidade": cidade,
            "Estado": estado,
            "CEP": cep,
            "Logradouro": logradouro,
        },
        "Dados Pessoais": {
            "Nome": nome_completo,
            "Idade": idade,
            "Email": email,
            "Telefone": telefone,
            "CPF": cpf,
            "Profissão": profissao,
            "Nome Social": nome_social
        },
        "Dados Financeiros": {
            "Renda Mensal": f"R$ {renda_mensal:.2f}",
            "Patrimônio": f"R$ {patrimonio:.2f}",
            "Dívidas": f"R$ {dividas:.2f}" if nome_sujo else "Sem dívidas",
            "Nome Sujo": "Sim" if nome_sujo else "Não",
        },
        "Dados Adicionais": {
            "Data de Nascimento": data_nascimento.strftime("%d/%m/%Y"),
            "Gênero": genero,
            "Estado Civil": estado_civil,
        },
        "Dados Bancários": {
            "Banco": banco,
            "Agência": agencia,
            "Conta": conta,
        },
        "Histórico de Compras": historico_compras,
        "Redes Sociais": redes_sociais,
    }

    # Gerando a personalidade com a API do Gemini (DEPOIS de criar o dicionário)
    personalidade = gerar_personalidade(informacoes)
    informacoes["Dados Pessoais"]["Personalidade"] = personalidade

    return informacoes


In [9]:
if __name__ == "__main__":
    dados_ficticios = obter_informacoes_ficticias()

    for secao, dados in dados_ficticios.items():
        print(f"\n**{secao}:**")
        if isinstance(dados, list):  # Verifica se é o histórico de compras
            for compra in dados:
                print(f"  - {compra}")
        else:
            for chave, valor in dados.items():
                print(f"  - {chave}: {valor}")


**Localização:**
  - Cidade: Camargo do Norte
  - Estado: Maranhão
  - CEP: 35030-497
  - Logradouro: Via de Vasconcelos

**Dados Pessoais:**
  - Nome: Laura das Neves
  - Idade: 30
  - Email: laura.neves@gmail.com
  - Telefone: 71 4791 1940
  - CPF: 250.976.183-86
  - Profissão: Armador
  - Nome Social: Pedro
  - Personalidade: Pedro, de 30 anos, é um indivíduo complexo e intrigante que vive em Camargo do Norte, Maranhão. Como armador, ele possui uma renda mensal confortável e um patrimônio considerável, o que lhe confere um certo nível de estabilidade financeira. No entanto, sob sua fachada segura e bem-sucedida, esconde-se uma história pessoal conturbada.

Pedro é uma pessoa reservada e introvertida. Ele tende a manter sua vida pessoal privada e raramente se abre com os outros. No entanto, quando estabelece confiança, ele pode ser surpreendentemente vulnerável. Sua personalidade é marcada por uma mistura de força e sensibilidade. Ele é resiliente e determinado, mas também é profund